# Srock Strategy - Industry Strategy 0002

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

### (1) Industry Data

In [2]:
sd = '2019-12-31'
ed = '2026-02-13'

In [3]:
sql_industry = f"""
SELECT
    date,
    industry_level1_code AS sw2021_level1,
    c_pct_rank(close / m_lag(close,  5)) AS _ret_05,
    c_pct_rank(close / m_lag(close, 20)) AS _ret_20,
    c_pct_rank(close / m_lag(close, 60)) AS _ret_60,
    c_rank(-1 * (_ret_05 + _ret_20 + _ret_60)) AS rank_ind
FROM cn_stock_industry_sw_bar1d JOIN cn_stock_industry_index_mapper on cn_stock_industry_sw_bar1d.instrument = cn_stock_industry_index_mapper.index_level1_code
QUALIFY rank_ind <= 5
ORDER BY date, rank_ind
"""

In [4]:
df_industry = dai.query(sql_industry, filters = {"date":[sd, ed]}).df()
df_industry

,date,sw2021_level1,rank_ind
0,2020-04-02,110000,1.0
1,2020-04-02,340000,2.0
2,2020-04-02,770000,3.0
3,2020-04-02,610000,4.0
4,2020-04-02,620000,5.0
...,...,...,...
6996,2026-02-13,510000,1.0
6997,2026-02-13,610000,2.0
6998,2026-02-13,720000,3.0
6999,2026-02-13,240000,4.5


### (2) Stock Data

In [5]:
sql_stock = f"""
SELECT 
    date, 
    instrument,
    sw2021_level1,
    -1 * (c_rank(roe_avg_lf) * 0.4 + c_rank(net_profit_to_parent_qoq_mrq) * 0.3 + c_rank(-1*pb) * 0.2) AS factor,
FROM cn_stock_prefactors
WHERE is_risk_warning = 0
AND list_sector IN (1, 2, 3)
AND list_days > 365
ORDER BY date
"""

In [6]:
df_stock = dai.query(sql_stock, filters = {"date":[sd, ed]}).df()
df_stock

,date,instrument,sw2021_level1,factor
0,2019-12-31,600898.SH,270000,-1912.0
1,2019-12-31,600198.SH,270000,-1651.3
2,2019-12-31,600083.SH,510000,-1803.4
3,2019-12-31,000017.SZ,280000,-1265.0
4,2019-12-31,002607.SZ,460000,-2061.9
...,...,...,...,...
6364076,2026-02-13,600581.SH,230000,-4277.7
6364077,2026-02-13,002547.SZ,280000,-3878.5
6364078,2026-02-13,300340.SZ,630000,-4126.3
6364079,2026-02-13,300169.SZ,220000,-3225.9


### (3) Combined Strategy Data

In [7]:
sql_combined = f"""
WITH
data_industry AS (
    {sql_industry}
),
data_stock AS (
    {sql_stock}
)
SELECT
    date,
    instrument,
    factor
FROM data_industry JOIN data_stock USING (date, sw2021_level1)
"""

In [8]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_combined}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 15
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE is_month_end_trade = 1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [9]:
df_strategy = dai.query(sql_trade, filters = {"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2020-04-30,600651.SH,-2804.4,1.0,0.066667
1,2020-04-30,600073.SH,-2742.3,2.0,0.066667
2,2020-04-30,300116.SZ,-2741.4,3.0,0.066667
3,2020-04-30,002270.SZ,-2712.7,4.0,0.066667
4,2020-04-30,002441.SZ,-2638.4,5.0,0.066667
...,...,...,...,...,...
1045,2026-01-30,002739.SZ,-3480.5,11.0,0.066667
1046,2026-01-30,601600.SH,-3461.4,12.0,0.066667
1047,2026-01-30,600668.SH,-3436.9,13.0,0.066667
1048,2026-01-30,002671.SZ,-3419.7,14.0,0.066667


## 3. Backtest Strategy

In [10]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""close""",
    order_price_field_sell="""close""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 18:50:09] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 18:50:09] [info     ] 2020-04-30, 2026-01-30, , , instruments=582
[2026-02-18 18:50:10] [info     ] bigtrader module V2.2.0
[2026-02-18 18:50:10] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
order_target_percent: price is null for '600651.SH' at '2020-04-30 15:00:00'
order_target_percent: price is null for '002531.SZ' at '2020-08-31 15:00:00'
order_target_percent: price is null for '000959.SZ' at '2021-08-31 15:00:00'
order_target_percent: price is null for '600213.SH' at '2024-04-30 15:00:00'
[2026-02-18 18:50:16] [info     ] backtest done, raw_perf_ds:dai.DataSource("_fecd7d67c5fe4cad872af6f152a1c3d1")


[2026-02-18 18:50:18] [info     ] bigtrader.v34 运行完成 [8.470s].
